# Scrapper de sites

> Récupère le contenu de chacun des URL classé par typologie (variable ndf dans le code), chargé depuis le fichier Sites.json.

> Stocke le résultat dans ndf qui conditionne la variable "fichier de sortie". Celle-ci peut être adaptée pour pointer sur une zone correcte de votre drive dans la zone "Personnalisation" dénotée ci-dessous. Les données des sites collectés sont stockées au format pickle dans le dossier /OUT/ContenusSites classé par typologie selon Sites.json.

> La variable de sortie est de type dictionnaire pour conserver les url sources (flexibilité pour pouvoir corriger). Cf. commentaires infra. 




In [ ]:
# Il faut initialiser le dossier à partir de GitHub 
# cette action est à faire une seule fois lors de la première utilisation
!git clone https://github.com/Patent2net/DistorsionCom.git

Cloning into 'DistorsionCom'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 198 (delta 60), reused 172 (delta 34), pack-reused 0
Receiving objects: 100% (198/198), 13.96 MiB | 14.94 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [ ]:
stockageEntree = "/content/DistorsionCom/OUT"
stockageSortie = "/content/DistorsionCom/OUT"


---
Personnalisation

> Ces cellules permettent de réaliser les traitements à partir de son propre espace de stockage. A n'exécuter que dans ce cas en adaptant les dossiers d'entrée et sortie. NE PAS EXECUTER SAUF A PERSONNALISER LES TRAITEMENTS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Récupérer les sorties sur son drive (créer un dossier "OUT")
stockageSortie = "/content/drive/MyDrive/OUT"

In [ ]:
# Récupérer les entrées sur son drive
stockageEntree = "/content/drive/MyDrive/OUT"

---

In [ ]:
import requests, re, pickle
from bs4 import BeautifulSoup

In [ ]:
import json
with open(stockageEntree + '/Ressources/Sites.json', 'r', encoding='utf-8') as f:
    CatSites = json.load(f)

# Première boucle pour lever les problèmes

> La dynamique du web fait que certains sites peuvent ne pas être accessibles, ne pas répondre à un instant t, ou faire planter le collecteur. Ce qui suit teste chaque URL et construit la variable BadUrl avec les urls en erreur.

In [ ]:
BadUrl = dict()
Done = []
for ndc in CatSites.keys():
  if ndc not in BadUrl.keys() and ndc not in Done:
    BadUrl[ndc] = []
    print(ndc)
    for url in CatSites[ndc][0]: 
      try:
        webpage = requests.get(url)
      except:
        print("bad", url)
        BadUrl[ndc].append(url)

In [ ]:
BadUrl [ndc]

In [ ]:
done = []  
# pourrait être initialisé par un os.listdir() sur le répertoire de sortie
     
for ndc in CatSites.keys():
  if ndc not in BadUrl.keys():
    BadUrl[ndc] = []
  if ndc not in done:
    soup = dict() # changement de type de données pour conserver l'URL source
  # récupère l'URL d'un site web et enregistre la page web
    for url in CatSites[ndc][0]: 
      if url not in BadUrl[ndc]:
        webpage = requests.get(url)
      else:
        pass
  # récupère le contenu de la page web à l'aide de BeautifulSoup
    #soup.append(BeautifulSoup(webpage.content, "html.parser"))
      tempoSoup = BeautifulSoup(webpage.content, "html.parser")
      if tempoSoup.title is not None:
        titre = tempoSoup.title.text 
      else:
        titre = ""

      texte = [x.get_text() for x in tempoSoup.find_all('p')]
      soup[url] = titre + '\n' + "\n".join(texte)
      # le contenu est dans le "casier" nommé par l'url
    # nettoyage
    for cle in soup.keys():
    # soup [cle] = re.sub('^a-zA-Z0-9àâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      re.sub('^a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      soup[cle] = soup[cle].replace("\xa0", " ")
      soup[cle] = soup[cle].replace("\n", " ")
      soup[cle] = soup[cle].replace("’", "'")
      soup[cle] = soup[cle].translate('utf8')
      dictionary = {"\\": ""}
      transtable= soup[cle].maketrans(dictionary)
      soup[cle] = soup[cle].translate(transtable)
      soup[cle] = str(soup[cle])
    
    # c'est pas tip top. Certains caractères restent. Variable selon les sites. 

    # sauvegarde sur Google Drive
    fichierDeSortie = stockageSortie + '/OUT/ContenusSites/' +CatSites [ndc][1] +'.pkl'
    with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans un pickle
      pickle.dump(soup, fictemp)
    done .append (ndc)

# Nettoyage

1. Il semble que le texte récupéré contienne de nombreux échappements d'apostrophes : `\'`
qui sont un "encodage" informatique particulier. Une substitution pour les remplacer par un apostrophe simple évitera que spacy s'embrouille.

2. BarUrl contient tous les url plantés rangés par catégorie. 

In [ ]:
import pprint as pp 

pp.pprint (BadUrl)

{'Hebergements': [],
 'Services': ['http://group-indigo.com', 'http://secretrip.pro'],
 'gouv': [],
 'locations': [],
 'mairies': [],
 'oTourism': [],
 'orga': [],
 'parcs': ['http://www.cevennes-parcnational.fr/fr',
           'http://www.mercantour-parcnational.fr/fr',
           'http://www.reunion-parcnational.fr/fr'],
 'restauration': []}


## Commentaires et exemple de ce qui est récupéré


*   les clés du dictionnaire de sortie sont les url
*   Le contenu du dictionnaire pour une clé donne le texte récupéré
* L'équivalent avec le précédent format (toutes les pages dans un texte) s'obtient facilement



In [ ]:
soup.keys()

dict_keys(['http://la-seyne.fr', 'http://le-pradet.fr', 'http://isere.fr', 'http://hyeres.fr', 'http://ville-lagarde.fr', 'http://toulon.fr', 'http://carqueiranne.fr', 'http://villedelacrau.fr'])

In [ ]:
soup ['http://la-seyne.fr']

"La Seyne.fr - Site officiel de la ville de La Seyne-sur-Mer Accueil   A Decrease font size. A Reset font size. A Increase font size.          Suivez l'actualité seynoise : portraits, dossiers, actions, événements… Toute l'information au plus près de chez vous.  A l'occasion de la Semaine européenne pour l'emploi des personnes handicapées (SEEPH), et sous l'impulsion de Valérie Guittienne, élue en charge de l'inclusion des personnes en situation de handicap, une matinée « Rencontres entreprises » s'est tenue le 16 novembre à Saint-Elme. Découvrez tous les événements culturels, sportifs, associatifs sur l'agenda officiel de la ville. Côté mer, vous découvrirez le célèbre rocher des “Deux Frères” depuis la plus grande plage de La Seyne “Les Sablettes”, ou bien vous plongerez dans ses eaux cristallines en vous initiant à la plongée sous-marine en toute sécurité ! Côté forêt, ce sont des paysages grandioses et préservés qui s'offriront à vous. La Corniche Merveilleuse qui vous permettra d'